Building likelihood maps for a set of representative images.

> TODO: use images from dataset and superimpose the bounding boxes on them


In [1]:
import retinoto_py as fovea
subset_factor = 50

In [2]:
resolution = (100, 100)
resolution = (30, 30)
resolution = (25, 25)
size_ratio = .4


# comparing datasets and models (fovea vs cartesian)

In [ ]:
from torchvision.transforms import v2 as transforms

for do_fovea in [True, False]:
    for dataset in fovea.params.all_datasets:
        args = fovea.Params(do_mask=not(do_fovea), do_fovea=do_fovea, 
                            batch_size=1, subset_factor=subset_factor)
        VAL_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'val'
        val_dataset = fovea.get_dataset(args, VAL_DATA_DIR, do_full_preprocess=False)
        val_loader = fovea.get_loader(args, val_dataset)

        if do_fovea:
            model_filename = args.data_cache /  f'32_fovea_model_name={args.model_name}_dataset=bbox.pth'
        else:
            model_filename = args.data_cache /  f'20_model_name={args.model_name}_dataset=bbox.pth'
        model = fovea.load_model(args, model_filename=model_filename)

        npy_filename = args.data_cache / f'43_likelihood_maps_dataset={dataset}_{do_fovea=}.npy'
        # %rm {npy_filename}  # FORCING RECOMPUTE
        if npy_filename.exists():
            results = fovea.np.load(npy_filename)
        else:
            n_dataset = len(val_dataset)
            with fovea.torch.no_grad():
                results = fovea.np.empty((resolution[0], resolution[1], n_dataset))
                for i_batch, (image, true_idx) in fovea.tqdm(enumerate(val_loader), total=n_dataset):
                    image, true_idx = image.to(args.device), true_idx.to(args.device)
                    crop_size = min(image.shape[1], image.shape[2])
                    image = transforms.CenterCrop(crop_size)(image)
                    pos_H, pos_W, probas = fovea.compute_likelihood_map(args, model, image.squeeze(0), size_ratio=size_ratio, resolution=resolution)
                    results[:, :, i_batch] = probas[:, true_idx].cpu().numpy().reshape(resolution)
                fovea.np.save(npy_filename, results)        
        results.min(), results.max()

  0%|          | 0/1000 [00:00<?, ?it/s]

# comparing effect of size

In [ ]:
from torchvision.transforms import v2 as transforms

dataset = 'bbox'
VAL_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'val'
val_dataset = fovea.get_dataset(args, VAL_DATA_DIR, do_full_preprocess=False)
val_loader = fovea.get_loader(args, val_dataset)
model_filename = args.data_cache /  f'32_fovea_model_name={args.model_name}_dataset=bbox.pth'
model = fovea.load_model(args, model_filename=model_filename)

for size_ratio_ in np.linspace(0, 1, 9, endpoint=True)[1:-1]:
    args = fovea.Params(do_fovea=True, batch_size=1, subset_factor=subset_factor)


    npy_filename = args.data_cache / f'43_likelihood_maps_{size_ratio_=}.npy'
    # %rm {npy_filename}  # FORCING RECOMPUTE
    if npy_filename.exists():
        results = fovea.np.load(npy_filename)
    else:
        n_dataset = len(val_dataset)
        with fovea.torch.no_grad():
            results = fovea.np.empty((resolution[0], resolution[1], n_dataset))
            for i_batch, (image, true_idx) in fovea.tqdm(enumerate(val_loader), total=n_dataset):
                image, true_idx = image.to(args.device), true_idx.to(args.device)
                crop_size = min(image.shape[1], image.shape[2])
                image = transforms.CenterCrop(crop_size)(image)
                pos_H, pos_W, probas = fovea.compute_likelihood_map(args, model, image.squeeze(0), size_ratio=size_ratio, resolution=resolution)
                results[:, :, i_batch] = probas[:, true_idx].cpu().numpy().reshape(resolution)
            fovea.np.save(npy_filename, results)        
    results.min(), results.max()